## Deliverable 2. Create a Customer Travel Destinations Map.

In [34]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [35]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_Database.csv")
city_data_df.head()

,Weather_Database,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,M,45.6667,4.0833,57.67,74,0,1.72,FR,clear sky
1,1,R,-23.1203,-134.9692,73.99,68,100,16.82,PF,overcast clouds
2,2,H,-42.8794,147.3294,53.46,64,40,12.66,AU,scattered clouds
3,3,U,-54.8000,-68.3000,40.66,81,75,23.02,AR,light rain
4,4,J,42.0970,-79.2353,72.99,73,100,8.05,US,overcast clouds


In [36]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 50
What is the maximun temperature you would like for your trip? 100


In [37]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,Weather_Database,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,M,45.6667,4.0833,57.67,74,0,1.72,FR,clear sky
1,1,R,-23.1203,-134.9692,73.99,68,100,16.82,PF,overcast clouds
2,2,H,-42.8794,147.3294,53.46,64,40,12.66,AU,scattered clouds
4,4,J,42.0970,-79.2353,72.99,73,100,8.05,US,overcast clouds
6,6,P,-21.3500,165.1500,75.09,62,100,7.61,NC,overcast clouds


In [38]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

641


Weather_Database       641
City                   641
Lat                    641
Lng                    641
Max Temp               641
Humidity               641
Cloudiness             641
Wind Speed             641
Country                636
Current Description    641
dtype: int64

In [39]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

636


,Weather_Database,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,M,45.6667,4.0833,57.67,74,0,1.72,FR,clear sky
1,1,R,-23.1203,-134.9692,73.99,68,100,16.82,PF,overcast clouds
2,2,H,-42.8794,147.3294,53.46,64,40,12.66,AU,scattered clouds
4,4,J,42.0970,-79.2353,72.99,73,100,8.05,US,overcast clouds
6,6,P,-21.3500,165.1500,75.09,62,100,7.61,NC,overcast clouds


In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,M,FR,57.67,clear sky,45.6667,4.0833,
1,R,PF,73.99,overcast clouds,-23.1203,-134.9692,
2,H,AU,53.46,scattered clouds,-42.8794,147.3294,
4,J,US,72.99,overcast clouds,42.0970,-79.2353,
6,P,NC,75.09,overcast clouds,-21.3500,165.1500,
7,S,PG,78.85,broken clouds,-10.6167,150.6667,
9,M,AU,73.17,clear sky,-20.7333,139.5000,
10,A,IR,74.93,scattered clouds,34.5217,50.0019,
12,N,IL,73.81,clear sky,31.4222,34.5886,
15,A,PF,77.47,light rain,-9.8000,-139.0333,


In [41]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.


NameError: name 'np' is not defined

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(15)

In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
